In [2]:
import pandas as pd

 # IMPORT DES JEUX DE DONNEES VELIB

In [ ]:
# Fonction qui traite les données d'une dataframe
# -> Conversion aux bons formats etc..

def traitement_data(file):
    
    print("Chargement du csv ... ")
    df = pd.read_csv(file, keep_default_na=False)
    print('\n')

    print("Traitement des données ... ")
    print('\n')
    
    print('Conversion datetime :')
    df['ping_time'] = pd.to_datetime(df['ping_time'].str.strip(), format='%d/%m/%Y %H:%M:%S')
    
    df['duedate'] = pd.to_datetime(df['duedate'].str.strip().str.replace('T', ' ')\
                                                .str.replace('+02', '' ).str.replace('+01', '' ), format='%Y-%m-%d %H:%M:%S:%f')
    
    print('Conversion coordonnees géographiques:')
    
    df['latitude'] =  df['coordonnees_geo'].str.strip().str.replace('[', ' ')\
        .str.replace(']', ' ').str.split(',', expand=True)[0].astype('float')
    
    df['longitude'] =  df['coordonnees_geo'].str.strip().str.replace('[', ' ')\
        .str.replace(']', ' ').str.split(',', expand=True)[1].astype('float')
    
    df = df.drop(columns='coordonnees_geo')
    
    print('Conversion bool')
    df['is_renting']     = df['is_renting'].str.strip().replace('NON', False)
    df['is_renting']     = df['is_renting'].str.strip().replace('OUI', True)
    df['is_renting']     = df['is_renting'].astype('bool')
    
    df['is_installed']   = df['is_installed'].str.strip().replace('NON', False)
    df['is_installed']   = df['is_installed'].str.strip().replace('OUI', True)
    df['is_installed']   = df['is_installed'].astype('bool')
    
    df['is_returning']   = df['is_returning'].str.strip().replace('NON', False)
    df['is_returning']   = df['is_returning'].str.strip().replace('OUI', True)
    df['is_returning']   = df['is_returning'].astype('bool')
    
    print("Conversion str")
    df['nom_arrondissement_communes'] = df['nom_arrondissement_communes'].str.strip().astype('string')
    df['name'] = df['name'].str.strip().astype('string')
   
    # station code contient des str, on les convertis en int
    print("Conversion int")
    
    if not(df['stationcode'].apply(type) == str).sum()==0:
        df.loc[(df['stationcode'].apply(type) == str), 'stationcode'] = \
            df.loc[(df['stationcode'].apply(type) == str), 'stationcode'].str.extract('(\d+)')[0].astype(float)
    df= df.dropna()
    df['stationcode'] = df['stationcode'].astype(int)
    
    # idem pour capacity
    if not(df['capacity'].apply(type) == str).sum()==0:
        df.loc[(df['capacity'].apply(type) == str), 'capacity'] = \
            df.loc[(df['capacity'].apply(type) == str), 'capacity'].str.extract('(\d+)')[0].astype(float)
    df= df.dropna()
    df['capacity'] = df['capacity'].astype(int)
    
    
    print("\n")

    
    return df


In [3]:
#df = pd.read_csv("C:\\Users\\kabah\\OneDrive\\Bureau\\lepont\\29 capstone 3\\2022 velib csv\\Velib_AUG_2022\\Velib_AUG_2022.csv")
#df

In [4]:
def checkFileExistance(filePath):

    try:

        with open(filePath, "r") as f:

            return True

    except FileNotFoundError as e:

        return False

    except IOError as e:

        return False



# liste des mois 

L = ['JAN', 'FEB', 'MAY', 'JUNE', 'JULY', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
 
# dataframe sous forme de dictionnaire

df = {}
 
# On parcours les années

for date in ['2022', '2023']:

    # Puis les mois listés dans L

    for mois in L:

        # file : nom du fichier cible

        file = 'C:\\Users\\kabah\\OneDrive\\Bureau\\velib csv\\Velib_'+mois+'_'+date+'.csv'   #"C:\Users\kabah\OneDrive\Bureau\velib csv\Velib_AUG_2022.csv""
 
        # je teste l'existance de file

        if checkFileExistance(file):

            # SI il est existe de stock le resultat dans le dicitnnaire

            df[mois+'_'+date] = pd.read_csv(file)

        # J'afffiche le mois et l'année pour savoir ou j'en suis dans la boucle

        print(date + ' ' + mois, checkFileExistance(file))



2022 JAN False
2022 FEB False
2022 MAY True
2022 JUNE True
2022 JULY True
2022 AUG True
2022 SEP False
2022 OCT False
2022 NOV True


C:\Users\kabah\AppData\Local\Temp\ipykernel_4180\368871283.py:45: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df[mois+'_'+date] = pd.read_csv(file)


2022 DEC True
2023 JAN True
2023 FEB True


C:\Users\kabah\AppData\Local\Temp\ipykernel_4180\368871283.py:45: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df[mois+'_'+date] = pd.read_csv(file)


2023 MAY True
2023 JUNE True
2023 JULY True
2023 AUG True
2023 SEP True
2023 OCT True
2023 NOV False
2023 DEC False


In [5]:
# name : string
# stationcode : int
# ebike (nombre vélo electrique dispo au moment du ping): int
# mechanical (nombre vélo mécanique dispo au moment du ping): int
# coordonnées_géo : lat, long : A SEPARER EN DEUX FLOAT
# duedate ??  : METTRE AU FORMAT DATE
# numbike available (nbr vélo disponible sur la borne au moment du ping): int 
# numdockavailable (nbr dock disponible sur la borne au moment du ping): int
# capacity (nombre de bornes de la station): A METTRE EN INT
# is renting (OUI: ya un vélo a louer. NON: ya pas de vélo dispo) : A METTRE EN BOOL
# is installed (OUI: la station est installée, NON elle ne l'est pas): A METTRE EN BOOL
# nim_arrondissement : Sring
# is returning (?????????): A METTRE EN BOOL
# Ping time (date a la quelle les données ont été mise a jour) : A METTRE AU FORMAT DATETOIME


In [6]:
# 14 fichiers csv table 14 colonnes 3561610 lignes 
# format : texte (string), int, date, 
# mai 2022

 # PRESENTATION DES DONNEES 

On a 14 datasets qui nous fournissent les données enregistrées dans les stations velib en région parisienne.
Chaque fichier représente les données d'un mois de l'année en 2022 et 2023.

MAI  2022

In [7]:
df['MAY_2022'].info() # 14 colonnes et 6200904 lignes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6200904 entries, 0 to 6200903
Data columns (total 14 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   name                         object 
 1   stationcode                  object 
 2   ebike                        int64  
 3   mechanical                   int64  
 4   coordonnees_geo              object 
 5   duedate                      object 
 6   numbikesavailable            int64  
 7   numdocksavailable            int64  
 8   capacity                     float64
 9   is_renting                   object 
 10  is_installed                 object 
 11  nom_arrondissement_communes  object 
 12  is_returning                 object 
 13  ping_time                    object 
dtypes: float64(1), int64(4), object(9)
memory usage: 662.3+ MB


In [8]:
df['MAY_2022'].isnull().sum()   # 15 valeurs nulles dans 4 colonnes (name, coordonnees_geo, 
                                  #capacity, nom_arrondissement_communes)

name                           15
stationcode                     0
ebike                           0
mechanical                      0
coordonnees_geo                15
duedate                         0
numbikesavailable               0
numdocksavailable               0
capacity                       15
is_renting                      0
is_installed                    0
nom_arrondissement_communes    15
is_returning                    0
ping_time                       0
dtype: int64

In [9]:
df['MAY_2022'].nunique() 

df['MAY_2022'].duplicated(subset=['ping_time']).sum()

6191127

In [10]:
df['MAY_2022'].duplicated().sum() # pas de lignes en doublon

0

JUIN 2022

In [11]:
df['JUNE_2022'].info() # 14 colonnes et 6131927 lignes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6131928 entries, 0 to 6131927
Data columns (total 14 columns):
 #   Column                       Dtype 
---  ------                       ----- 
 0   name                         object
 1   stationcode                  object
 2   ebike                        int64 
 3   mechanical                   int64 
 4   coordonnees_geo              object
 5   duedate                      object
 6   numbikesavailable            int64 
 7   numdocksavailable            int64 
 8   capacity                     int64 
 9   is_renting                   object
 10  is_installed                 object
 11  nom_arrondissement_communes  object
 12  is_returning                 object
 13  ping_time                    object
dtypes: int64(5), object(9)
memory usage: 655.0+ MB


In [12]:
df['JUNE_2022'].isnull().sum() # pas de valeurs nulles

name                           0
stationcode                    0
ebike                          0
mechanical                     0
coordonnees_geo                0
duedate                        0
numbikesavailable              0
numdocksavailable              0
capacity                       0
is_renting                     0
is_installed                   0
nom_arrondissement_communes    0
is_returning                   0
ping_time                      0
dtype: int64

In [13]:
df['JUNE_2022'].nunique()

name                             1420
stationcode                      1420
ebike                              55
mechanical                         73
coordonnees_geo                  1429
duedate                        302085
numbikesavailable                  84
numdocksavailable                  95
capacity                           66
is_renting                          2
is_installed                        2
nom_arrondissement_communes        60
is_returning                        2
ping_time                        8910
dtype: int64

In [14]:
df['JUNE_2022'].duplicated().sum() # pas de lignes en doublon

0

JUILLET 2022

In [15]:
df['JULY_2022'].info() # 14 colonnes et 6332842 lignes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6332843 entries, 0 to 6332842
Data columns (total 14 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   name                         object 
 1   stationcode                  object 
 2   ebike                        int64  
 3   mechanical                   int64  
 4   coordonnees_geo              object 
 5   duedate                      object 
 6   numbikesavailable            int64  
 7   numdocksavailable            int64  
 8   capacity                     float64
 9   is_renting                   object 
 10  is_installed                 object 
 11  nom_arrondissement_communes  object 
 12  is_returning                 object 
 13  ping_time                    object 
dtypes: float64(1), int64(4), object(9)
memory usage: 676.4+ MB


In [16]:
df['JULY_2022'].isnull().sum() # 2 valeurs nulles dans 4 colonnes (name, coordonnees_geo, 
                                  #capacity, nom_arrondissement_communes)

name                           2
stationcode                    0
ebike                          0
mechanical                     0
coordonnees_geo                2
duedate                        0
numbikesavailable              0
numdocksavailable              0
capacity                       2
is_renting                     0
is_installed                   0
nom_arrondissement_communes    2
is_returning                   0
ping_time                      0
dtype: int64

In [17]:
df['JULY_2022'].nunique()

name                             1419
stationcode                      1421
ebike                              60
mechanical                         72
coordonnees_geo                  1447
duedate                        317847
numbikesavailable                  85
numdocksavailable                  72
capacity                           63
is_renting                          2
is_installed                        2
nom_arrondissement_communes        60
is_returning                        2
ping_time                        5971
dtype: int64

In [18]:
df['JULY_2022'].duplicated().sum() # pas de lignes en doublon

0

AOUT 2022

In [19]:
df['AUG_2022'].info() # 14 colonnes et 6132309 lignes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6132310 entries, 0 to 6132309
Data columns (total 14 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   name                         object 
 1   stationcode                  object 
 2   ebike                        int64  
 3   mechanical                   int64  
 4   coordonnees_geo              object 
 5   duedate                      object 
 6   numbikesavailable            int64  
 7   numdocksavailable            int64  
 8   capacity                     float64
 9   is_renting                   object 
 10  is_installed                 object 
 11  nom_arrondissement_communes  object 
 12  is_returning                 object 
 13  ping_time                    object 
dtypes: float64(1), int64(4), object(9)
memory usage: 655.0+ MB


In [20]:
df['AUG_2022'].isnull().sum()   # 4 valeurs nulles dans 4 colonnes (name, coordonnees_geo, 
                                  #capacity, nom_arrondissement_communes)

name                           4
stationcode                    0
ebike                          0
mechanical                     0
coordonnees_geo                4
duedate                        0
numbikesavailable              0
numdocksavailable              0
capacity                       4
is_renting                     0
is_installed                   0
nom_arrondissement_communes    4
is_returning                   0
ping_time                      0
dtype: int64

In [21]:
df['AUG_2022'].nunique()

name                             1420
stationcode                      1423
ebike                              58
mechanical                         73
coordonnees_geo                  1435
duedate                        312446
numbikesavailable                  82
numdocksavailable                  69
capacity                           63
is_renting                          2
is_installed                        2
nom_arrondissement_communes        61
is_returning                        2
ping_time                        4997
dtype: int64

In [22]:
df['AUG_2022'].duplicated().sum() # pas de ligne en doublon

0

NOVEMBRE 2022

In [23]:
df['NOV_2022'].info()   # 14 colonnes et 3561610 lignes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3561611 entries, 0 to 3561610
Data columns (total 14 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   name                         object 
 1   stationcode                  object 
 2   ebike                        int64  
 3   mechanical                   int64  
 4   coordonnees_geo              object 
 5   duedate                      object 
 6   numbikesavailable            int64  
 7   numdocksavailable            int64  
 8   capacity                     float64
 9   is_renting                   object 
 10  is_installed                 object 
 11  nom_arrondissement_communes  object 
 12  is_returning                 object 
 13  ping_time                    object 
dtypes: float64(1), int64(4), object(9)
memory usage: 380.4+ MB


In [24]:
df['NOV_2022'].isnull().sum()  # 9 valeurs nulles dans 4 colonnes (name, coordonnees_geo, 
                                  #capacity, nom_arrondissement_communes)

name                           9
stationcode                    0
ebike                          0
mechanical                     0
coordonnees_geo                9
duedate                        0
numbikesavailable              0
numdocksavailable              0
capacity                       9
is_renting                     0
is_installed                   0
nom_arrondissement_communes    9
is_returning                   0
ping_time                      0
dtype: int64

In [25]:
df['NOV_2022'].nunique()

name                             1430
stationcode                      1432
ebike                              53
mechanical                         73
coordonnees_geo                  1446
duedate                        300688
numbikesavailable                  84
numdocksavailable                  71
capacity                           63
is_renting                          2
is_installed                        2
nom_arrondissement_communes        61
is_returning                        2
ping_time                        2558
dtype: int64

In [26]:
df['NOV_2022'].duplicated().sum()  # pas de lignes en doublon

0

DECEMBRE 2022

In [27]:
df['DEC_2022'].info()      # 14 colonnes et  2584754 lignes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2584755 entries, 0 to 2584754
Data columns (total 14 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   name                         object 
 1   stationcode                  object 
 2   ebike                        int64  
 3   mechanical                   int64  
 4   coordonnees_geo              object 
 5   duedate                      object 
 6   numbikesavailable            int64  
 7   numdocksavailable            int64  
 8   capacity                     float64
 9   is_renting                   object 
 10  is_installed                 object 
 11  nom_arrondissement_communes  object 
 12  is_returning                 object 
 13  ping_time                    object 
dtypes: float64(1), int64(4), object(9)
memory usage: 276.1+ MB


In [28]:
df['DEC_2022'].isnull().sum()    # 4 valeurs nulles dans 4 colonnes (name, coordonnees_geo, 
                                  #capacity, nom_arrondissement_communes)

name                           4
stationcode                    0
ebike                          0
mechanical                     0
coordonnees_geo                4
duedate                        0
numbikesavailable              0
numdocksavailable              0
capacity                       4
is_renting                     0
is_installed                   0
nom_arrondissement_communes    4
is_returning                   0
ping_time                      0
dtype: int64

In [29]:
df['DEC_2022'].nunique()

name                             1430
stationcode                      2857
ebike                              48
mechanical                         71
coordonnees_geo                  1436
duedate                        154546
numbikesavailable                  79
numdocksavailable                  68
capacity                           62
is_renting                          2
is_installed                        2
nom_arrondissement_communes        61
is_returning                        2
ping_time                        4142
dtype: int64

In [30]:
df['DEC_2022'].duplicated().sum() # pas de lignes en doublon

0

In [31]:
df['DEC_2022']['ping_time'].value_counts()

08/12/2022 16:01:00    1427
08/12/2022 02:46:01    1427
08/12/2022 04:36:01    1427
08/12/2022 04:26:01    1427
08/12/2022 04:16:01    1427
                       ... 
08/01/2023 03:40:01       5
08/01/2023 03:30:01       5
08/01/2023 03:20:01       5
08/01/2023 03:10:01       5
17/01/2023 09:20:01       5
Name: ping_time, Length: 4142, dtype: int64

JANVIER 2023

In [32]:
df['JAN_2023'].info() # 14 colonnes et 6208766 lignes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6208767 entries, 0 to 6208766
Data columns (total 14 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   name                         object 
 1   stationcode                  int64  
 2   ebike                        int64  
 3   mechanical                   int64  
 4   coordonnees_geo              object 
 5   duedate                      object 
 6   numbikesavailable            int64  
 7   numdocksavailable            int64  
 8   capacity                     float64
 9   is_renting                   object 
 10  is_installed                 object 
 11  nom_arrondissement_communes  object 
 12  is_returning                 object 
 13  ping_time                    object 
dtypes: float64(1), int64(5), object(8)
memory usage: 663.2+ MB


In [33]:
df['JAN_2023'].isnull().sum() # 12 valeurs nulles dans 4 colonnes (name, coordonnees_geo, 
                                  #capacity, nom_arrondissement_communes)

name                           12
stationcode                     0
ebike                           0
mechanical                      0
coordonnees_geo                12
duedate                         0
numbikesavailable               0
numdocksavailable               0
capacity                       12
is_renting                      0
is_installed                    0
nom_arrondissement_communes    12
is_returning                    0
ping_time                       0
dtype: int64

In [34]:
df['JAN_2023'].nunique()

name                             1430
stationcode                      1432
ebike                              50
mechanical                         78
coordonnees_geo                  1451
duedate                        313891
numbikesavailable                  83
numdocksavailable                  71
capacity                           63
is_renting                          2
is_installed                        2
nom_arrondissement_communes        61
is_returning                        2
ping_time                        4406
dtype: int64

In [35]:
df['JAN_2023'].duplicated().sum() # pas de lignes en dopublon

0

FEVRIER 2023

In [36]:
df['FEB_2023'].info() # 14 colonnes et 5045089 lignes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5045090 entries, 0 to 5045089
Data columns (total 14 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   name                         object 
 1   stationcode                  int64  
 2   ebike                        int64  
 3   mechanical                   int64  
 4   coordonnees_geo              object 
 5   duedate                      object 
 6   numbikesavailable            int64  
 7   numdocksavailable            int64  
 8   capacity                     float64
 9   is_renting                   object 
 10  is_installed                 object 
 11  nom_arrondissement_communes  object 
 12  is_returning                 object 
 13  ping_time                    object 
dtypes: float64(1), int64(5), object(8)
memory usage: 538.9+ MB


In [37]:
df['FEB_2023'].isnull().sum()  # 2 valeurs nulles dans 4 colonnes (name, coordonnees_geo, 
                                  #capacity, nom_arrondissement_communes)

name                           2
stationcode                    0
ebike                          0
mechanical                     0
coordonnees_geo                2
duedate                        0
numbikesavailable              0
numdocksavailable              0
capacity                       2
is_renting                     0
is_installed                   0
nom_arrondissement_communes    2
is_returning                   0
ping_time                      0
dtype: int64

In [38]:
df['FEB_2023'].duplicated().sum() # pas de lignes en doublon

0

In [39]:
df['FEB_2023'].nunique()

name                             1428
stationcode                      1429
ebike                              60
mechanical                         77
coordonnees_geo                  1442
duedate                        249596
numbikesavailable                  83
numdocksavailable                  70
capacity                           62
is_renting                          2
is_installed                        2
nom_arrondissement_communes        61
is_returning                        2
ping_time                        9783
dtype: int64

MAI 2023

In [40]:
df['MAY_2023'].info() # 14 colonnes et 6596287 lignes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6596288 entries, 0 to 6596287
Data columns (total 14 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   name                         object 
 1   stationcode                  object 
 2   ebike                        int64  
 3   mechanical                   int64  
 4   coordonnees_geo              object 
 5   duedate                      object 
 6   numbikesavailable            int64  
 7   numdocksavailable            int64  
 8   capacity                     float64
 9   is_renting                   object 
 10  is_installed                 object 
 11  nom_arrondissement_communes  object 
 12  is_returning                 object 
 13  ping_time                    object 
dtypes: float64(1), int64(4), object(9)
memory usage: 704.6+ MB


In [41]:
df['MAY_2023'].isnull().sum()  # 5 valeurs nulles dans 4 colonnes (name, coordonnees_geo, 
                                  #capacity, nom_arrondissement_communes)

name                           5
stationcode                    0
ebike                          0
mechanical                     0
coordonnees_geo                5
duedate                        0
numbikesavailable              0
numdocksavailable              0
capacity                       5
is_renting                     0
is_installed                   0
nom_arrondissement_communes    5
is_returning                   0
ping_time                      0
dtype: int64

In [42]:
df['MAY_2023'].duplicated().sum() # pas de lignes en doublon

0

In [43]:
df['MAY_2023'].nunique()

name                             1436
stationcode                      2870
ebike                              62
mechanical                         72
coordonnees_geo                  1452
duedate                        313219
numbikesavailable                  85
numdocksavailable                  69
capacity                           62
is_renting                          2
is_installed                        2
nom_arrondissement_communes        62
is_returning                        2
ping_time                        4686
dtype: int64

JUIN 2023

In [44]:
df['JUNE_2023'].info()   # 14 colonnes et 4647743 lignes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4647744 entries, 0 to 4647743
Data columns (total 14 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   name                         object 
 1   stationcode                  int64  
 2   ebike                        int64  
 3   mechanical                   int64  
 4   coordonnees_geo              object 
 5   duedate                      object 
 6   numbikesavailable            int64  
 7   numdocksavailable            int64  
 8   capacity                     float64
 9   is_renting                   object 
 10  is_installed                 object 
 11  nom_arrondissement_communes  object 
 12  is_returning                 object 
 13  ping_time                    object 
dtypes: float64(1), int64(5), object(8)
memory usage: 496.4+ MB


In [45]:
df['JUNE_2023'].isnull().sum()  # 2 valeurs nulles dans 4 colonnes (name, coordonnees_geo, 
                                  #capacity, nom_arrondissement_communes)

name                           2
stationcode                    0
ebike                          0
mechanical                     0
coordonnees_geo                2
duedate                        0
numbikesavailable              0
numdocksavailable              0
capacity                       2
is_renting                     0
is_installed                   0
nom_arrondissement_communes    2
is_returning                   0
ping_time                      0
dtype: int64

In [46]:
df['JUNE_2023'].duplicated().sum() # pas de lignes en doublon

0

In [47]:
df['JUNE_2023'].nunique()

name                             1442
stationcode                      1444
ebike                             199
mechanical                        203
coordonnees_geo                  1450
duedate                        219544
numbikesavailable                 227
numdocksavailable                  72
capacity                           64
is_renting                          2
is_installed                        2
nom_arrondissement_communes        62
is_returning                        2
ping_time                        3863
dtype: int64

JUILLET 2023

In [48]:
df['JULY_2023'].info()  # 14 colonnes et 6691869 lignes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6691870 entries, 0 to 6691869
Data columns (total 14 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   name                         object 
 1   stationcode                  int64  
 2   ebike                        int64  
 3   mechanical                   int64  
 4   coordonnees_geo              object 
 5   duedate                      object 
 6   numbikesavailable            int64  
 7   numdocksavailable            int64  
 8   capacity                     float64
 9   is_renting                   object 
 10  is_installed                 object 
 11  nom_arrondissement_communes  object 
 12  is_returning                 object 
 13  ping_time                    object 
dtypes: float64(1), int64(5), object(8)
memory usage: 714.8+ MB


In [49]:
df['JULY_2023'].isnull().sum() # 36 valeurs nulles dans 4 colonnes (name, coordonnees_geo, 
                                  #capacity, nom_arrondissement_communes)

name                           36
stationcode                     0
ebike                           0
mechanical                      0
coordonnees_geo                36
duedate                         0
numbikesavailable               0
numdocksavailable               0
capacity                       36
is_renting                      0
is_installed                    0
nom_arrondissement_communes    36
is_returning                    0
ping_time                       0
dtype: int64

In [50]:
df['JULY_2023'].duplicated().sum() # pas de lignes en doublon

0

In [51]:
df['JULY_2023'].nunique()

name                             1451
stationcode                      1452
ebike                              58
mechanical                         72
coordonnees_geo                  1480
duedate                        330578
numbikesavailable                  85
numdocksavailable                  71
capacity                           63
is_renting                          2
is_installed                        2
nom_arrondissement_communes        62
is_returning                        2
ping_time                       12714
dtype: int64

AOUT 2023

In [52]:
df['AUG_2023'].info()  # 14 colonnes et 6635860 lignes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6635861 entries, 0 to 6635860
Data columns (total 14 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   name                         object 
 1   stationcode                  int64  
 2   ebike                        int64  
 3   mechanical                   int64  
 4   coordonnees_geo              object 
 5   duedate                      object 
 6   numbikesavailable            int64  
 7   numdocksavailable            int64  
 8   capacity                     float64
 9   is_renting                   object 
 10  is_installed                 object 
 11  nom_arrondissement_communes  object 
 12  is_returning                 object 
 13  ping_time                    object 
dtypes: float64(1), int64(5), object(8)
memory usage: 708.8+ MB


In [53]:
df['AUG_2023'].isnull().sum()   # 13 valeurs nulles dans 4 colonnes (name, coordonnees_geo, 
                                  #capacity, nom_arrondissement_communes)

name                           13
stationcode                     0
ebike                           0
mechanical                      0
coordonnees_geo                13
duedate                         0
numbikesavailable               0
numdocksavailable               0
capacity                       13
is_renting                      0
is_installed                    0
nom_arrondissement_communes    13
is_returning                    0
ping_time                       0
dtype: int64

In [54]:
df['AUG_2023'].duplicated().sum() # pas de lignes en doublon

0

In [55]:
df['AUG_2023'].nunique()

name                             1441
stationcode                      1442
ebike                              73
mechanical                         76
coordonnees_geo                  1458
duedate                        337022
numbikesavailable                  97
numdocksavailable                  71
capacity                           64
is_renting                          2
is_installed                        2
nom_arrondissement_communes        62
is_returning                        2
ping_time                        8285
dtype: int64

SEPTEMBRE 2023

In [56]:
df['SEP_2023'].info() # 14 colonnes et 6280390 lignes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6280391 entries, 0 to 6280390
Data columns (total 14 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   name                         object 
 1   stationcode                  float64
 2   ebike                        float64
 3   mechanical                   float64
 4   coordonnees_geo              object 
 5   duedate                      object 
 6   numbikesavailable            int64  
 7   numdocksavailable            int64  
 8   capacity                     float64
 9   is_renting                   object 
 10  is_installed                 object 
 11  nom_arrondissement_communes  object 
 12  is_returning                 object 
 13  ping_time                    object 
dtypes: float64(4), int64(2), object(8)
memory usage: 670.8+ MB


In [57]:
df['SEP_2023'].isnull().sum() # 21 valeurs nulles dans 4 colonnes (name, coordonnees_geo, 
                                  #capacity, nom_arrondissement_communes)

name                           21
stationcode                     1
ebike                           1
mechanical                      1
coordonnees_geo                21
duedate                         0
numbikesavailable               0
numdocksavailable               0
capacity                       21
is_renting                      0
is_installed                    0
nom_arrondissement_communes    21
is_returning                    0
ping_time                       0
dtype: int64

In [58]:
df['SEP_2023'].duplicated().sum()

0

In [59]:
df['SEP_2023'].nunique()

name                             1442
stationcode                      1443
ebike                              70
mechanical                         74
coordonnees_geo                  1446
duedate                        313037
numbikesavailable                 105
numdocksavailable                  75
capacity                           63
is_renting                          2
is_installed                        2
nom_arrondissement_communes        62
is_returning                        2
ping_time                        6339
dtype: int64

OCTOBRE 2023

In [60]:
df['OCT_2023'].info()   # 14 colonnes et 1458121 lignes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458122 entries, 0 to 1458121
Data columns (total 14 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   name                         1458121 non-null  object 
 1   stationcode                  1458122 non-null  int64  
 2   ebike                        1458122 non-null  int64  
 3   mechanical                   1458122 non-null  int64  
 4   coordonnees_geo              1458121 non-null  object 
 5   duedate                      1458122 non-null  object 
 6   numbikesavailable            1458122 non-null  int64  
 7   numdocksavailable            1458122 non-null  int64  
 8   capacity                     1458121 non-null  float64
 9   is_renting                   1458122 non-null  object 
 10  is_installed                 1458122 non-null  object 
 11  nom_arrondissement_communes  1458121 non-null  object 
 12  is_returning                 1458122 non-n

In [61]:
df['OCT_2023'].duplicated().sum() # pas de ligne en doublon

0

In [62]:
df['OCT_2023'].nunique()

name                            1433
stationcode                     1436
ebike                             58
mechanical                        87
coordonnees_geo                 1438
duedate                        72691
numbikesavailable                 98
numdocksavailable                 76
capacity                          64
is_renting                         2
is_installed                       2
nom_arrondissement_communes       62
is_returning                       2
ping_time                       2132
dtype: int64

In [66]:
df['OCT_2023'].describe()

,stationcode,ebike,mechanical,numbikesavailable,numdocksavailable,capacity
count,1.458122e+06,1.458122e+06,1.458122e+06,1.458122e+06,1.458122e+06,1.458121e+06
mean,1.859761e+04,2.898886e+00,5.927998e+00,8.826885e+00,2.157281e+01,3.157744e+01
std,1.218500e+04,3.331086e+00,7.959044e+00,9.482628e+00,1.353260e+01,1.237467e+01
min,1.001000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.102500e+04,1.000000e+00,1.000000e+00,2.000000e+00,1.300000e+01,2.300000e+01
50%,1.610100e+04,2.000000e+00,3.000000e+00,5.000000e+00,2.000000e+01,2.900000e+01
75%,2.195100e+04,4.000000e+00,8.000000e+00,1.300000e+01,2.900000e+01,3.700000e+01
max,9.200800e+04,9.700000e+01,1.660000e+02,2.630000e+02,2.010000e+02,2.010000e+02


# PROFILING

Nous sommes en présence de 14 fichiers plats csv contenant chacune une table dont les colonnes sont séparées par des ",".

Ce dataset contient les informations des stations Vélib dans les environs de Paris, mises à jour toutes les 10 minutes, et sur plusieurs mois (mai, juin, juillet, août, novembre et décembre) en 2022 et (janvier, février, mai, juin, juillet, août, septembre, octobre) en 2023. Ces données sont mises à disposition au public pour suivre l'évolution du service en temps réel.

Ces 14 fichiers sont constitués de 14 colonnes et plus de 6 millions de lignes avec une clé primaire correspondant à la colonne "station code", pour le reste des colonnes, on a :

- 'name' : nom de la station, format string texte,

- 'stationcode' : numéro unique d’identification de la station. Ce numéro identifie la station au sein du service Vélib’ Métropole, format int(entier),
   
- 'ebike' : nombre vélo electrique dispo au moment du ping, format int(entier),

- 'mechanical' : nombre vélo mécanique dispo au moment du ping, format string, 

- 'coordonnees_geo' : longitude de la station et latitude de la station, format string

- 'duedate': Date actualisation de la données (instant T), format string à convertir en format date

- 'numbikesavailable': nombre de vélos disponibles, format int,

- 'numdocksavailable': nombre de bornettes disponibles, format int,
  
- 'capacity' : nombre de bornettes dans la station, format int,
  
- 'is_renting': variable binaire indiquant si la station peut louer des vélos (is_renting=oui si le statut de la station est opérationnel), format string,
  
- 'is_installed' :  variable binaire indiquant si la station a déjà été déployée (oui) ou est encore en cours de déploiement (non), format string,
  
- -'is_returning': variable binaire indiquant si la station peut recevoir des vélos (is_renting=oui si le statut de la station est opérationnel), format string,
  
- 'ping_time' (# last_reported : date de la dernière mise-à-jour ), format string,

Dans la préparation qui suit, nous allons traiter chacune des tables pour sécuriser et nettoyer les jeux de données.    



# Traitement des données


In [ ]:
import os
 
# On applique la fonction traitement_data sur chaque df et on enregistre le resultat sous forme de CSV.

directory = 'dataset'

i=0
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        print(i)
        print(filename+':')
        df = traitement_data(f)
        df.to_csv('Data\\Data_velib\\data_entieres\\'+filename)
        
        print('\n')
        i+=1
